In [1]:
from __future__ import division
import sys

# Third-Party Libraries
from numpy import *
import numpy as np
from numpy.fft import fft

In [2]:
def ATH(f):
    return 3.64*np.float_power(f/1000, -0.8) - \
        6.5*np.exp(-0.6*np.power(f/1000 - 3.3, 2)) + \
        np.float_power(10, -3)*np.power(f/1000, 4)

def bark(f):
    return 13*np.arctan(0.76*f/1000) + 3.5*np.arctan(f/1000/7.5)**2

In [13]:
#
# Constants
# ------------------------------------------------------------------------------
#
df = 44100.0
dt = 1.0 / df
N = 1152
# N = MPEG.N
# M = MPEG.M
   
k = arange(N // 2 + 1)
f_k = k * df / N
b_k = bark(f_k)

k_i = r_[0:49, 49:97:2, 97:251:4]
f_i = k_i * df / N
b_i = bark(f_i)
ATH_i = ATH(f_i)
subband_i = array([int(s) for s in round_(f_i * 32 / (0.5 * df) - 0.5)])
k

/home/sweet/2-coursework/434eces/labs/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in float_power
  This is separate from the ipykernel package so we can avoid doing imports until


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [4]:
#
# Perceptual Model
# ------------------------------------------------------------------------------
#
def P_k_from_frame(frame, window=hanning, dB=True):
    alpha = 1.0 / np.sqrt(sum(window(N)**2) / N)
    frame = alpha * window(N) * frame

    frame_fft_2 = abs(fft(frame)) ** 2

    P_k = 2.0 * frame_fft_2[:(N // 2 + 1)] / N ** 2
    P_k[0] = 0.5 * P_k[0]
    if (N % 2 == 0):
        P_k[-1] = 0.5 * P_k[-1]

    P_k = 10.0 ** (96.0 / 10.0) * P_k
    if dB:
        P_k = 10.0 * np.log10(P_k)  
    return P_k

class Classifier(object):
    def __init__(self):
        small  = array([-2, +2])
        medium = array([-3, -2, +2, +3]) 
        large  = array([-6, -5, -4, -3, -2, +2, +3, +4, +5, +6])
        self.neighbourhood = 256 * [None]
        for _k in range(2, 63):
            self.neighbourhood[_k] = small
        for _k in range(63, 127):
            self.neighbourhood[_k] = medium
        for _k in range(127, 251):
            self.neighbourhood[_k] = large        
    def __call__(self, k, P):
        assert all(k == arange(0, N // 2 + 1))
        k_t = []
        P_t = []
        for _k in arange(3, 251):
            if (P[_k-1] <= P[_k] and P[_k+1] <= P[_k]):
                js = self.neighbourhood[_k]
                if all(P[_k] >= 5.0 * P[_k+js]):
                    k_t.append(_k)
                    P_t.append(P[_k-1] + P[_k] + P[_k+1])
                    P[_k-1] = P[_k] = P[_k+1] = 0.0
        return (array(k_t), array(P_t)), (k, P)        

classify = Classifier()

def group_by_critical_band(k, P):
    b_k = bark(f_k)
    cb_k = array([int(b) for b in floor(b_k)])
    bands = [[[], []] for _ in arange(amax(cb_k) + 1)]
    for _k, _P in zip(k, P):
        band = bands[cb_k[_k]]
        band[0].append(_k)
        band[1].append(_P)
    for b, band in enumerate(bands):
        bands[b] = array(band)
    return bands

def merge_tonals(k_t, P_t):
    bands = group_by_critical_band(k_t, P_t)
    k_t_out, P_t_out = [], []
    for band, k_P_s in enumerate(bands):
        if len(k_P_s[0]):
            k_max = None
            P_max = - inf 
            for _k, _P in zip(*k_P_s):
               if _P > P_max:
                   k_max = _k
                   P_max = _P
            k_t_out.append(k_max)
            P_t_out.append(P_max)
    return array(k_t_out), array(P_t_out)

def merge_non_tonals(k_nt, P_nt):
    bands = group_by_critical_band(k_nt, P_nt)
    k_nt_out = zeros(len(bands), dtype=uint8)
    P_nt_out = zeros(len(bands))
    for band, k_P_s in enumerate(bands):
        if len(k_P_s[0]):
            k, P = k_P_s
            P_sum = sum(P)
            if P_sum == 0.0:
                P = ones_like(P)
            k_mean = int(round(average(k, weights=P))) 
            k_nt_out[band] = k_mean
            P_nt_out[band] = P_sum
    return k_nt_out, P_nt_out

def threshold(k, P):
    ATH_k = 10 ** (ATH(f_k) / 10.0)
    k_out, P_out = [], []
    for (_k, _P) in zip(k, P):
        if _P > ATH_k[_k]:
            k_out.append(_k)
            P_out.append(_P)
    return array(k_out), array(P_out)

def maskers(frame, merge=True, ATH_threshold=True):
    P_k = P_k_from_frame(frame)
    (k_t, P_t), (k_nt, P_nt) = classify(k, P_k)
    if merge:
        k_t, P_t = merge_tonals(k_t, P_t)
        k_nt, P_nt = merge_non_tonals(k_nt, P_nt)
    if ATH_threshold:
        k_t, P_t = threshold(k_t, P_t)
        k_nt, P_nt = threshold(k_nt, P_nt)
    return (k_t, P_t), (k_nt, P_nt)

def excitation_pattern(b, b_m, I_m, tonal):
    db = b - b_m
    db_1 = minimum(db + 1.0, 0.0)
    db_2 = minimum(db      , 0.0)
    db_3 = maximum(db      , 0.0)
    db_4 = maximum(db - 1.0, 0.0)    
    mask  = I_m \
          + (11.0 - 0.40 * I_m) * db_1 \
          + ( 6.0 + 0.40 * I_m) * db_2 \
          - (17.0             ) * db_3 \
          + (       0.15 * I_m) * db_4
    if tonal:
        mask += -1.525 - 0.275 * b - 4.5
    else:
        mask += -1.525 - 0.175 * b - 0.5
    return mask

def mask_from_frame(frame, subband=True):
    frame = array(frame, copy=False)
    if shape(frame) != (N,):
        error = "the frame should a 1-dim. array of length {0}."
        raise TypeError(error.format(N))
        
    mask_i = 10.0 ** (ATH_i / 10.0)
    
    (k_t, P_t), (k_nt, P_nt) = maskers(frame)
    for masker_index in arange(len(k_t)):
        _b, _P = b_k[k_t[masker_index]], P_t[masker_index]
        ep = excitation_pattern(b_i, b_m=_b, I_m=10.0*log10(_P), tonal=True)
        mask_i += 10.0 ** (ep / 10.0)
    for masker_index in arange(len(k_nt)):
        _b, _P = b_k[k_nt[masker_index]], P_nt[masker_index]
        ep = excitation_pattern(b_i, b_m=_b, I_m=10.0*log10(_P), tonal=False) 
        mask_i += 10.0 ** (ep / 10.0)
    mask_i = 10.0 * log10(mask_i)

    subband_mask = [[] for _ in range(32)]
    for i, _mask_i in enumerate(mask_i):
        subband_mask[subband_i[i]].append(_mask_i)
    for i, _masks in enumerate(subband_mask):
        subband_mask[i] = amin(_masks)
    return array(subband_mask)

In [8]:
# def mask_from_frame(frame, subband=True):
frame = array(frame, copy=False)
mask_i = 10.0 ** (ATH_i / 10.0)

(k_t, P_t), (k_nt, P_nt) = maskers(frame)
P_k = P_k_from_frame(frame)
#     for masker_index in arange(len(k_t)):
#         _b, _P = b_k[k_t[masker_index]], P_t[masker_index]
#         ep = excitation_pattern(b_i, b_m=_b, I_m=10.0*log10(_P), tonal=True)
#         mask_i += 10.0 ** (ep / 10.0)
#     for masker_index in arange(len(k_nt)):
#         _b, _P = b_k[k_nt[masker_index]], P_nt[masker_index]
#         ep = excitation_pattern(b_i, b_m=_b, I_m=10.0*log10(_P), tonal=False) 
#         mask_i += 10.0 ** (ep / 10.0)
#     mask_i = 10.0 * log10(mask_i)

#     subband_mask = [[] for _ in range(32)]
#     for i, _mask_i in enumerate(mask_i):
#         subband_mask[subband_i[i]].append(_mask_i)
#     for i, _masks in enumerate(subband_mask):
#         subband_mask[i] = amin(_masks)
#     return array(subband_mask)

/home/sweet/2-coursework/434eces/labs/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in float_power
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
P_k_from_frame(frame)

array([  22.93917074,   26.44521741,   27.90319251,   30.26217453,
         33.48682353,   37.62633037,   42.88055896,   49.73836721,
         59.44690861,   77.15696835,   90.08250576,   89.49311073,
         74.43767836,   58.24249701,   48.88104934,   42.1099854 ,
         36.76615485,   32.33687367,   28.5464563 ,   25.22851313,
         22.27458676,   19.60994558,   17.18087629,   14.94746429,
         12.87923525,   10.95239368,    9.14800296,    7.45074574,
          5.84805585,    4.3294955 ,    2.88629922,    1.51103354,
          0.1973391 ,   -1.06026789,   -2.26655021,   -3.42567166,
         -4.54129421,   -5.61665632,   -6.65463642,   -7.65780497,
         -8.62846747,   -9.56870005,  -10.48037943,  -11.36520799,
        -12.2247351 ,  -13.0603752 ,  -13.87342333,  -14.66506841,
        -15.4364048 ,  -16.18844222,  -16.92211448,  -17.63828704,
        -18.33776367,  -19.02129234,  -19.68957034,  -20.34324891,
        -20.98293729,  -21.60920633,  -22.22259176,  -22.82359

In [6]:
    t = r_[0:1152] * dt
#     try:
#         f = float(sys.argv[1])
#     except:
#         print ("need an argument: the tone frequency f in Hz")
#         sys.exit(1)
    f = 400.0
    frame = sin(2 * pi * f * t)

    print ()
    print ("Subband Mask Levels for a pure tone with SPL = 96 dB, f={0}".format(f))
    print (80*"-")
    for i, mask_level in enumerate(mask_from_frame(frame)):
        print ("subband {0:2d}: {1!r} dB".format(i, mask_level))
    print ()


Subband Mask Levels for a pure tone with SPL = 96 dB, f=400.0
--------------------------------------------------------------------------------


/home/sweet/2-coursework/434eces/labs/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in float_power
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: zero-size array to reduction operation minimum which has no identity

In [16]:
r_